In [1]:
import os
import subprocess
import pandas as pd
import h5py
import numpy as np

In [2]:
def filter_tn5_motifs(motifs_meme_file):
    """
    Filters TN5 motifs from the given MEME file and writes the filtered motifs to a new MEME file.
    Adds the necessary header to the MEME file.
    
    :param motifs_meme_file: Path to the MEME file containing motifs to be filtered.
    :return: Path to the filtered MEME file.
    """
    filtered_motifs = []
    
    # Step 1: Load motifs from the MEME file
    with open(motifs_meme_file, 'r') as f:
        motif_data = []
        for line in f:
            if line.startswith("MOTIF"):  # Start of a new motif
                if motif_data:
                    filtered_motifs.append(motif_data)
                motif_data = [line.strip()]  # Start a new motif
            else:
                motif_data.append(line.strip())  # Add data to current motif
        
        # Don't forget to add the last motif
        if motif_data:
            filtered_motifs.append(motif_data)
    
    # Step 2: Filter motifs - placeholder for your filtering logic
    # Example: Add a simple check, e.g., filtering based on motif name containing "TN5"
    filtered_motifs = [motif for motif in filtered_motifs if "TN5" in motif[0]]  # Adjust this filter as needed
    
    # Step 3: Write the MEME header and filtered motifs
    filtered_motifs_file = motifs_meme_file.replace(".meme", "_filtered.meme")
    
    with open(filtered_motifs_file, 'w') as f:
        # Write MEME file header
        f.write("MEME version 4\n")
        f.write("ALPHABET= ACGT\n")
        f.write("strands: + -\n")
        f.write("Background letter frequencies\n")
        f.write("A 0.25 C 0.25 G 0.25 T 0.25\n")
        
        # Add an extra newline after the header
        f.write("\n")
        
        # Write filtered motifs
        for motif in filtered_motifs:
            for line in motif:
                f.write(line + "\n")
    
    print(f"Filtered MEME file written to: {filtered_motifs_file}")
    
    return filtered_motifs_file


In [3]:
def decode_seqlet(seqlet_one_hot):
    """
    Decode the sequence from one-hot encoding. 
    Assumes the encoding is A=0, C=1, G=2, T=3.
    :param seqlet_one_hot: The one-hot encoded sequence (2D array of shape [sequence_length, 4]).
    :return: The decoded sequence as a string.
    """
    mapping = ['A', 'C', 'G', 'T']
    # For each row (which represents a nucleotide), find the index with value 1 (the base)
    return ''.join([mapping[seqlet.argmax()] for seqlet in seqlet_one_hot])


In [4]:
def write_meme_file(motifs, output_path):
    """
    Writes motifs to a MEME file.
    :param motifs: List of tuples [(motif_name, pwm), ...]
    :param output_path: Path to the output .meme file.
    """
    with open(output_path, 'w') as f:
        f.write("MEME version 5.0\n\n")
        f.write("ALPHABET= ACGT\n\n")
        f.write("strands: + -\n\n")
        f.write("Background letter frequencies:\n")
        f.write("A 0.25 C 0.25 G 0.25 T 0.25\n\n")
        
        for motif_name, pwm in motifs:
            f.write(f"MOTIF {motif_name}\n")
            f.write(f"letter-probability matrix: alength= 4 w= {pwm.shape[0]}\n")
            for row in pwm:
                f.write(" ".join(f"{prob:.6f}" for prob in row) + "\n")
            f.write("\n")


In [6]:
def run_tomtom(query_meme_path, target_meme_path, output_dir):
    """
    Run TOMTOM to compare motifs.
    :param query_meme_path: Path to the query motif file in MEME format.
    :param target_meme_path: Path to the target motif database in MEME format.
    :param output_dir: Path to the output directory to save TOMTOM results.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    command = [
        "tomtom",
        "-oc", output_dir,
        query_meme_path,
        target_meme_ path
    ]
    try:
        print(f"Calling TOMTOM command {command}")
        subprocess.run(command, check=True)
        print(f"TOMTOM completed. Results saved in {output_dir}")
    except subprocess.CalledProcessError as e:
        print(f"Error running TOMTOM: {e}")

In [7]:
def extract_pwm_from_contrib_scores(contrib_scores):
    """
    Converts contribution scores to a position weight matrix (PWM).
    :param contrib_scores: numpy array of shape (L, 4) where L is the sequence length.
    :return: PWM as a numpy array of the same shape.
    """
    # Ensure contrib_scores is a numpy array
    contrib_scores = np.array(contrib_scores)
    
    # Normalize scores to obtain probabilities
    pwm = np.exp(contrib_scores)  # Exponentiate scores
    pwm = pwm / np.sum(pwm, axis=1, keepdims=True)  # Normalize to sum to 1 at each position
    
    return pwm

In [30]:
def filter_and_copy_patterns(input_file, output_dir, group_type, base_logo_dir, threshold, motifs_meme_file):
    """
    Filters motifs based on TOMTOM results and copies the filtered motifs.
    :param input_file: Path to the input file (HDF5 file) containing motif data.
    :param output_dir: Directory to save filtered motifs.
    :param group_type: Type of motifs to process (e.g., 'neg_patterns' or 'pos_patterns').
    :param base_logo_dir: Directory containing motif logo images.
    :param threshold: Threshold for filtering motifs based on TOMTOM p-value.
    :param motifs_meme_file: Path to the MEME file containing TN5 motifs.
    """
    # Step 1: Filter TN5 motifs from the given MEME file
    print(f"Filtering TN5 motifs from {motifs_meme_file}...")
    try:
        tn5_filtered_file = filter_tn5_motifs(motifs_meme_file)
        print(f"Filtered TN5 motifs saved to {tn5_filtered_file}")
    except Exception as e:
        print(f"Error filtering TN5 motifs: {e}")
        return

    # Define paths for query and target MEME files
    query_meme_path = os.path.join(base_logo_dir, f"{group_type}.meme")
    print(f"query_meme_path is {query_meme_path}")
    tomtom_output_dir = os.path.join(output_dir, group_type)
    print(f"tomtom_output_dir is {tomtom_output_dir}")

    # Step 2: Read patterns from HDF5 and write them to MEME format (query motifs)
    motifs = []
    try:
        with h5py.File(input_file, 'r') as h5_file:
            if group_type not in h5_file:
                raise KeyError(f"Group '{group_type}' not found in HDF5 file.")

            patterns_group = h5_file[group_type]

            # Iterate through each pattern and decode seqlets
            for pattern_name in patterns_group:
                try:
                    pattern_group = patterns_group[pattern_name]
                    print(f"Inspecting pattern group for {pattern_name}: {pattern_group}")

                    # Print all keys in the pattern_group for inspection
                    # for key in pattern_group.keys():
                    #     print(f"Key: {key}, Value: {pattern_group[key]}")

                    print(f"Processing pattern: {group_type}/{pattern_name}")

                    # Check if the pattern has seqlets
                    seqlets_group = pattern_group.get('seqlets')
                    if seqlets_group is None:
                        print(f"Seqlets not found for pattern {pattern_name}. Skipping.")
                        continue

                    contrib_scores = pattern_group['contrib_scores'][()]
                    # print(f"contrib_scores is : {contrib_scores}")
                    pwm = extract_pwm_from_contrib_scores(contrib_scores)
                    # print(f"pwm is : {pwm}")
                    motifs.append((pattern_name, pwm))
                    # Print the motifs list after appending the new pattern and seqlet
                    # print(f"Current motifs list: {motifs}")

                except KeyError as e:
                    print(f"Missing key for pattern {pattern_name}: {e}")
                except Exception as e:
                    print(f"Error processing pattern {pattern_name}: {e}")

        # Step 3: Write the query motifs to MEME format
        if motifs:
            # print(f"Motifs to write are: {motifs}")
            print(f"query_meme_path is: {query_meme_path}")
            write_meme_file(motifs, query_meme_path)
            print(f"Query motifs written to MEME file at {query_meme_path}")
        else:
            print(f"No valid motifs found for {group_type}. Skipping TOMTOM.")
            return

    except Exception as e:
        print(f"Error reading from HDF5 file: {e}")
        return

    # Step 4: Run TOMTOM to compare the query and target motifs
    try:
        print(f"Running TOMTOM for {group_type} motifs...")
        run_tomtom(query_meme_path, tn5_filtered_file, tomtom_output_dir)
    except Exception as e:
        print(f"Error running TOMTOM: {e}")
        return

    # Step 5: Parse TOMTOM results and filter based on p-value threshold
    tomtom_results_path = os.path.join(tomtom_output_dir, "tomtom.tsv")
    print(f"tomtom_results_path is {tomtom_results_path}")
    
    try:
        tomtom_results = pd.read_csv(
            tomtom_results_path,
            sep="\t",
            comment="#",
            usecols=["Query_ID", "Target_ID", "p-value", "q-value", "E-value"]
        )
    except FileNotFoundError:
        print(f"TOMTOM results not found for {group_type}. Skipping.")
        return
    except Exception as e:
        print(f"Error reading TOMTOM results: {e}")
        return

    # Filter motifs based on the p-value threshold
    filtered_motifs = tomtom_results[tomtom_results["p-value"] <= (threshold / 100.0)]

    if filtered_motifs.empty:
        print(f"No motifs passed the threshold for {group_type}.")
        return

    print(f"Filtered motifs for {group_type}:")
    print(filtered_motifs)

    # Step 6: Copy filtered motifs' logo images to output directory
    for motif_id in filtered_motifs["Query_ID"].unique():
        src_path = os.path.join(base_logo_dir, f"{motif_id}.png")
        dest_path = os.path.join(tomtom_output_dir, f"{motif_id}.png")

        # Check if the logo image exists before copying
        if os.path.exists(src_path):
            subprocess.run(["cp", src_path, dest_path])
            print(f"Copied {motif_id}.png to {dest_path}")
        else:
            print(f"Logo image for {motif_id} not found. Skipping.")

    print(f"Filtered motifs for {group_type} have been processed and copied.")

    # Step 7: Save filtered motifs as a CSV file for later reference
    filtered_motifs_csv = os.path.join(tomtom_output_dir, f"{group_type}_filtered_motifs.csv")
    filtered_motifs.to_csv(filtered_motifs_csv, index=False)
    print(f"Filtered motifs saved to {filtered_motifs_csv}")

    print(f"Motif processing for {group_type} is complete.")

In [31]:
# Path to provided motifs.meme file
motifs_meme_file = "./steps_inputs/step6_3/motifs.meme"

# Threshold for motif filtering
threshold = 80  # Adjust as necessary

# Input file path
input_file_path = "/scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/auxiliary/interpret_subsample/modisco_results_profile_scores.h5"

# Extracting SPECIES, ENCSR_id, and ENCFF_id from input_file_path (optional, if you want to extract dynamically)
# Using hardcoded values as given
SPECIES = "human"
ENCSR_id = "ENCSR004DZS"
ENCFF_id = "ENCFF709MRG"

# Base directory for logos and output
base_logo_dir = f"/scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/{SPECIES}/{ENCSR_id}/{ENCFF_id}/fold_0/step62.bpnetPipeline/evaluation/modisco_profile/trimmed_logos/"
output_dir = f"/scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/{SPECIES}/{ENCSR_id}/{ENCFF_id}/fold_0/step62.bpnetPipeline/qc/out_step_6_3_2_tomtom"


In [32]:
base_logo_dir

'/scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/evaluation/modisco_profile/trimmed_logos/'

In [33]:
# tn5_filtered_file

In [34]:
# Define the group types
group_types = ['neg_patterns', 'pos_patterns']

# Loop through the group types and run the processing
for group_type in group_types:
    print(f"Processing {group_type} with threshold {threshold}...")
    
    # Call the function directly from the notebook
    filter_and_copy_patterns(input_file_path, output_dir, group_type, base_logo_dir, threshold, motifs_meme_file)


Processing neg_patterns with threshold 80...
Filtering TN5 motifs from ./steps_inputs/step6_3/motifs.meme...
Filtered MEME file written to: ./steps_inputs/step6_3/motifs_filtered.meme
Filtered TN5 motifs saved to ./steps_inputs/step6_3/motifs_filtered.meme
query_meme_path is /scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/evaluation/modisco_profile/trimmed_logos/neg_patterns.meme
tomtom_output_dir is /scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/qc/out_step_6_3_2_tomtom/neg_patterns
Inspecting pattern group for pattern_0: <HDF5 group "/neg_patterns/pattern_0" (8 members)>
Processing pattern: neg_patterns/pattern_0
Inspecting pattern group for pattern_1: <HDF5 group "/neg_patterns/pattern_1" (5 members)>
Processing pattern: neg_patterns/pattern_1
query_meme_path is: /scratch/groups/akundaje/eil

Provide at least 50 motifs for accurate p-value computation.
The output directory '/scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/qc/out_step_6_3_2_tomtom/neg_patterns' already exists.
Its contents will be overwritten.
Processing query 1 out of 2 
# Computing q-values.
#   Cannot estimate pi_0 accurately from fewer than 100 p-values.
#   Total p-values = 16. Using pi_zero = 1.0.
Processing query 2 out of 2 
# Computing q-values.
#   Cannot estimate pi_0 accurately from fewer than 100 p-values.
#   Total p-values = 16. Using pi_zero = 1.0.


TOMTOM completed. Results saved in /scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/qc/out_step_6_3_2_tomtom/neg_patterns
tomtom_results_path is /scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/qc/out_step_6_3_2_tomtom/neg_patterns/tomtom.tsv
Filtered motifs for neg_patterns:
    Query_ID Target_ID   p-value   E-value   q-value
0  pattern_0     TN5_1  0.000130  0.001038  0.001117
1  pattern_1     TN5_1  0.000141  0.001124  0.001124
Logo image for pattern_0 not found. Skipping.
Logo image for pattern_1 not found. Skipping.
Filtered motifs for neg_patterns have been processed and copied.
Filtered motifs saved to /scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/qc/out_step_6_3_2_tomtom/neg_patterns/neg_pat

Provide at least 50 motifs for accurate p-value computation.
The output directory '/scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/qc/out_step_6_3_2_tomtom/pos_patterns' already exists.
Its contents will be overwritten.
Processing query 1 out of 19 
# Computing q-values.
#   Cannot estimate pi_0 accurately from fewer than 100 p-values.
#   Total p-values = 16. Using pi_zero = 1.0.
Processing query 2 out of 19 
# Computing q-values.
#   Cannot estimate pi_0 accurately from fewer than 100 p-values.
#   Total p-values = 16. Using pi_zero = 1.0.
Processing query 3 out of 19 
# Computing q-values.
#   Cannot estimate pi_0 accurately from fewer than 100 p-values.
#   Total p-values = 16. Using pi_zero = 1.0.
Processing query 4 out of 19 
# Computing q-values.
#   Cannot estimate pi_0 accurately from fewer than 100 p-values.
#   Total p-values = 16. Using pi_zero = 1.0.
Processing query 5 out of 1

TOMTOM completed. Results saved in /scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/qc/out_step_6_3_2_tomtom/pos_patterns
tomtom_results_path is /scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/qc/out_step_6_3_2_tomtom/pos_patterns/tomtom.tsv
Filtered motifs for pos_patterns:
      Query_ID Target_ID   p-value   E-value   q-value
0    pattern_0     TN5_1  0.000133  0.001061  0.001061
1    pattern_1     TN5_1  0.000134  0.001075  0.001075
2   pattern_10     TN5_1  0.000132  0.001058  0.001138
3   pattern_11     TN5_1  0.000133  0.001062  0.001062
4   pattern_12     TN5_1  0.000134  0.001073  0.001073
5   pattern_13     TN5_1  0.000133  0.001063  0.001063
6   pattern_14     TN5_1  0.000133  0.001063  0.001063
7   pattern_15     TN5_1  0.000132  0.001058  0.001139
8   pattern_16     TN5_1  0.000133  

# Computing q-values.
#   Cannot estimate pi_0 accurately from fewer than 100 p-values.
#   Total p-values = 16. Using pi_zero = 1.0.


In [18]:
!cp /scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/evaluation/modisco_profile/trimmed_logos/pos_patterns.meme .

In [24]:
!cp /scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/evaluation/modisco_profile/trimmed_logos/neg_patterns.meme .

In [12]:
!ls /scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/evaluation/modisco_profile/trimmed_logos/

neg_patterns.meme		     pos_patterns.pattern_17.cwm.fwd.png
neg_patterns.pattern_0.cwm.fwd.png   pos_patterns.pattern_17.cwm.rev.png
neg_patterns.pattern_0.cwm.rev.png   pos_patterns.pattern_18.cwm.fwd.png
neg_patterns.pattern_1.cwm.fwd.png   pos_patterns.pattern_18.cwm.rev.png
neg_patterns.pattern_1.cwm.rev.png   pos_patterns.pattern_1.cwm.fwd.png
pos_patterns.meme		     pos_patterns.pattern_1.cwm.rev.png
pos_patterns.pattern_0.cwm.fwd.png   pos_patterns.pattern_2.cwm.fwd.png
pos_patterns.pattern_0.cwm.rev.png   pos_patterns.pattern_2.cwm.rev.png
pos_patterns.pattern_10.cwm.fwd.png  pos_patterns.pattern_3.cwm.fwd.png
pos_patterns.pattern_10.cwm.rev.png  pos_patterns.pattern_3.cwm.rev.png
pos_patterns.pattern_11.cwm.fwd.png  pos_patterns.pattern_4.cwm.fwd.png
pos_patterns.pattern_11.cwm.rev.png  pos_patterns.pattern_4.cwm.rev.png
pos_patterns.pattern_12.cwm.fwd.png  pos_patterns.pattern_5.cwm.fwd.png
pos_patterns.pattern_12.cwm.rev.png  pos_patterns.pattern_5.cwm.rev.png
pos_patterns.p

In [13]:
!cp /scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/evaluation/modisco_profile/trimmed_logos/pos_patterns.meme .

# cp pos_patterns.meme

In [26]:
!ls /scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/qc/out_step_6_3_2_tomtom/neg_patterns



neg_patterns_filtered_motifs.csv  tomtom.html  tomtom.tsv  tomtom.xml


In [29]:
!cp /scratch/groups/akundaje/eila/encode_pseudobulks/old_encode_pseudobulks_model_training/human/ENCSR004DZS/ENCFF709MRG/fold_0/step62.bpnetPipeline/qc/out_step_6_3_2_tomtom/neg_patterns/* .


